# Lexical Substitution

In this notebook we will be exploring the task of finding a suitable substitution for a target word in a sentence. It describes various methods that we have tried so that our program attempts to detect the right word out of 10 guesses.

## lexsub: default program

In [45]:
from lexsub import *

import os, sys, optparse
from tqdm import tqdm_notebook as tqdm
import pymagnitude
import subprocess
import logging

from lexsub import *
from lexsub_check import precision
import os
import re

from copy import deepcopy
from pymagnitude import *
from nltk.corpus import stopwords as nltk_stopwords
from functools import reduce
from collections import defaultdict

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Imports done successfully")

DATA = '../data'

INFO:root:Imports done successfully


## Run the default solution on dev

In [46]:
lexsub = LexSub(os.path.join(DATA,'glove.6B.100d.magnitude'))
output = []
with open(os.path.join(DATA,'input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexsub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))

sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner
sides edge bottom front club line both back place corner


## Evaluate the default output

In [11]:
with open(os.path.join(DATA,'reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=27.89


# Ideas Explored to improve the score -

### Idea 1. Implemented **[Retrofitting word vectors with semantic lexicons](https://www.aclweb.org/anthology/N15-1184/)** technique to achieve baseline score.

`Model Defination -` Information about word senses can be found in many semantic lexicons. The most widely used hand-curated semantic lexicon for the English language is the Wordnet ontology.

We use the semantic relations to augment the word representations we have in our pre-trained word vectors using retrofitting. We use the semantic relations from the Wordnet and modify the word vectors to use that information. This can make the word vectors more useful for lexical substitution which depends on knowledge of various senses of the target word.

The objective of retrofitting is to use the ontology graph in order to learn a matrix Q=(q1,…,qn) such that the columns of the matrix Q are close (in vector space) to the word vectors in Q̂  (so qi is close to q̂ i) and at the same time the columns of the matrix Q are close (in vector space) to the word vectors of other words that are adjacent vertices in the graph. So if (wi,wj) are connected by an edge in the ontology then we want qi and qj to be close in vector space. Retrofitting involves combining these two criteria to modify the word vectors for the words in our vocabulary.

### \begin{equation*} q_i = \frac{\sum _{j:(i,j)\in E}\beta _{ij}q_j + \alpha_i\hat{q}_i}{\sum _{j:(i,j)\in E}\beta _{ij} + \alpha_i} \end{equation*}



## Retrofit: baseline solution

In [48]:
# Function to generate retrofitted Embedding
def retrofit(wvecs, lexicon, numIters, alpha, beta):
    wordVecs = dict(wvecs)
    newWordVecs = deepcopy(wordVecs)
    wvVocab = set(newWordVecs.keys())
    loopVocab = wvVocab.intersection(set(lexicon.keys()))
    for it in range(numIters):
        # loop through every node also in ontology (else just use data estimate)
        for word in loopVocab:
            wordNeighbours = set(lexicon[word]).intersection(wvVocab)
            numNeighbours = len(wordNeighbours)
            #no neighbours, pass - use data estimate
            if numNeighbours == 0:
                continue
            newVec = numNeighbours * wordVecs[word]
            # loop over neighbours and add to new vector (currently with weight 1)
            for ppWord in wordNeighbours:
                newVec += newWordVecs[ppWord]
            newWordVecs[word] = newVec/(2*numNeighbours)
    return newWordVecs

In [49]:
# Write word vectors to file
def print_word_vecs(wordVectors, outFileName):
    sys.stderr.write('\nWriting down the vectors in '+outFileName+'\n')
    outFile = open(outFileName, 'w')  
    for word in wordVectors:
        outFile.write(word+' ')
        for val in wordVectors[word]:
            outFile.write('%.4f' %(val)+' ')
        outFile.write('\n')      
    outFile.close()

In [50]:
import re, os

isNumber = re.compile(r'\d+.*')

def norm_word(word):
    if isNumber.search(word.lower()):
        return '---num---'
    elif re.sub(r'\W+', '', word) == '':
        return '---punc---'
    else:
        return word.lower()

# Function to read lexicon file
def read_lexicon(filename):
    lexicon = {}
    for line in open(filename, 'r'):
        words = line.lower().strip().split()
        lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
    return lexicon

In [52]:
# generate retrofitted text file
topn = 10
lexicon = read_lexicon(os.path.join(DATA, 'lexicons', 'wordnet-synonyms+.txt'))
lexsub = LexSub(DATA + '/glove.6B.100d.magnitude', topn)
newLexsub = retrofit(lexsub.wvecs, lexicon, topn, 1.0, 1.0)
print_word_vecs(newLexsub, DATA + '/glove.6B.100d.retrofit.txt')


Writing down the vectors in ../data/glove.6B.100d.retrofit.txt


In [53]:
# generate the magnitude file
subprocess.run(["python3", "-m", "pymagnitude.converter", "-i", DATA + '/glove.6B.100d.retrofit.txt', "-o", DATA + '/glove.6B.100d.retrofit.magnitude'])


CompletedProcess(args=['python3', '-m', 'pymagnitude.converter', '-i', '../data/glove.6B.100d.retrofit.txt', '-o', '../data/glove.6B.100d.retrofit.magnitude'], returncode=0)

## Run the retrofitted baseline solution on dev

In [54]:
lexsub = LexSub(os.path.join(DATA,'glove.6B.100d.retrofit.magnitude'))
output = []
with open(os.path.join(DATA,'input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexsub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))

edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along
edge bottom front line corner back left way then along


## Evaluate the retrofitted baseline output

Achieved **Dev Score 46.56 using wordnet-synonyms.txt file** and **Dev Score 40.52 using wordnet-synonyms+.txt** file.

In [55]:
with open(os.path.join(DATA,'reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=40.52


### Idea 2. Modify retroffiting technique to imporve baseline score

<!--
- Implemented the retrofitting technique to achieve baseline score.
- Achieved different Dev Score using different lexicon files. Achieved Dev Score ~46 using wordnet-synonyms.txt file and Dev Score 41.52 using wordnet-synonyms+.txt file.
- Implemented A Simple Word Embedding Model for Lexical Substitution to extract better word substitution. This technique didn't improve result rather decrease the score.
- Experimented with different number of candidate match for the target word. Lower the number better the score.
- Experimented with different window size for context words around target word. Best score was achieved for 3.
- Experimented with using left window (i.e. context words to the left of target word) for context words. Similar score as with using both left and right window.
- Experimented with using glove.6B.100d.retrofit.magnitude for calculating  𝑐𝑜𝑠(𝑠,𝑡)  and glove.6B.100d.magnitude for calculatng  𝑐𝑜𝑠(𝑠,𝑐) . Did not result in any improvement in score. -->

Folowing are the modifications to original retrofitting implementation.

- The original paper **[Retrofitting word vectors with semantic lexicons](https://www.aclweb.org/anthology/N15-1184/)**, all words in lexicon are used to minimize the eucledian distance. Instead of doing that, we calculate the cosine similarity for each of them and pick only the top N based on the similarity scores. The following code shows it for N=10 and we obtained a **score of 43.57**

- We also normalized words in lexicon by discarding numbers and punctuations.

- We also evaluated for different values of alpha and beta in the equation. Following code shows alpha = num of neighbours in the lexicon and beta is set to a contant 1.

In [34]:
def copyPymagnitude(wv):
    return deepcopy({k: v for k, v in wv})

# Implementing changes to the baseline retrofitting implementation to improve score
def retrofit(wv, lexicon, T):
    qHat = copyPymagnitude(wv)
    q = copyPymagnitude(wv)
    vocabQHat = set([k for k, v in wv])
    loopOn = vocabQHat.intersection(set(lexicon.keys()))
    for t in range(T):
        for word in loopOn:
            wordNeighbours = set(lexicon[word]).intersection(vocabQHat)
            cand_scores = []
            for x in wordNeighbours:
                cand_scores.append(get_substitutability(x, word))
            # pick only top N based on the similarity scores
            sorted_candidates = sorted(zip(list(wordNeighbours), cand_scores), key=lambda x: x[1], reverse=True)
            wordNeighbours = [sub for sub, score in sorted_candidates][:10]
            numNeighbours = len(wordNeighbours)
            if numNeighbours == 0:
                continue
            newVec = numNeighbours * qHat[word]
            for ppWord in wordNeighbours:
                newVec += q[ppWord]
            q[word] = newVec / (2 * numNeighbours)
    return q


In [36]:
output_vector_filename = "glove.6B.100d.retrofit.similar10.{}"
generate_filename = DATA + '/' + output_vector_filename.format("magnitude")
wv = Magnitude(DATA + "/glove.6B.100d.magnitude")

isNumber = re.compile(r'\d+.*')

def norm_word(word):
    if isNumber.search(word.lower()):
        return '---num---'
    elif re.sub(r'\W+', '', word) == '':
        return '---punc---'
    else:
        return word.lower()

def build_lexicon(filename):
    lexicon = {}
    for line in open(filename, 'r'):
        words = line.lower().strip().split()
        lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
    return lexicon


def get_substitutability(t, s):
    score = wv.similarity(t, s)
    return score

lexicon = build_lexicon(os.path.join(DATA, "lexicons", "wordnet-synonyms+.txt"))
q = retrofit(wv, lexicon, 10)


In [37]:
# generate text file
output_vector_text_file = DATA + "/" + output_vector_filename.format("txt")
with open(output_vector_text_file, "w") as f:
    for i, (k, v) in enumerate(q.items()):
        line = " ".join([k] + list(map(str, v)) + ["\n"])
        f.write(line)

# generate the magnitude file
subprocess.run(["python3", "-m", "pymagnitude.converter", "-i", output_vector_text_file, "-o", generate_filename])

CompletedProcess(args=['python3', '-m', 'pymagnitude.converter', '-i', '../data/glove.6B.100d.retrofit.similar10.txt', '-o', '../data/glove.6B.100d.retrofit.similar10.magnitude'], returncode=0)

## Run the modifed retrofitted solution on dev

In [41]:
lexsub = LexSub(os.path.join(DATA, output_vector_filename.format("magnitude")))
output = []

# Run on the retrofit file
with open(os.path.join(DATA,'input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        output.append(" ".join(lexsub.substitutes(int(fields[0].strip()), fields[1].strip().split())))
print("\n".join(output[:10]))


front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way
front bottom back edge line corner left place right way


## Evaluate the modified retrofitted output

Achieved **Dev Score 43.57 using wordnet-synonyms+.txt** file.

In [44]:
# Check score
with open(os.path.join(DATA,'reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]

print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=43.57


## Idea 3. Incorporating context words

After retrofitting, we incorporate the context around the target word to find better substitute words. We tried various substitutability measures; Add, BalAdd, Mult, BalMult.
We experiment with removing the stop words, changing the number of candidates words, the scope of the context around the target. We also tried variants of substitutability measures where we replaced the cosine similarity with a distance metric to give the measure. However nothing worked well for us and we always got scores below 43%. The following code block shows one such implementation with a score of **40%**. 

## Training with context

As part of retrofitting, we ignored any context of word while calculating similarity. In the following process, we implemented a context based solution inspired from [A Simple Word Embedding Model for Lexical Substitution](https://www.aclweb.org/anthology/W15-1501/) paper.

We experimented with a context vector generated from the dev.txt file. We retrofit the previously retrofitted vector on a window of lenght N for a given word in dev.txt. For example:

_key: side, sentence: this will help the younger generation to know and understand better its neighbors living just on the other side of the mediterranean_

In this case, we try to bring the words on a window (2 words before and 2 words after "side" in the example) closer to the word "side" with the assumption that words with same context are surrounded by similar words. With this assumption, synonym word "edge" would get a higher similarity for a context that contains similar words as in this window.


In [8]:
stopwords = list(set(nltk_stopwords.words('english')))

def get_words(s, index, window=5, left_window=False):
    """ Extract a list of words from a sentence string with punctuation, spaces etc 
    s = sentence 
    """
    # strip punctuation 
    s = re.sub(r'[^\w\s]','',s)
    # replace newline 
    s = s.replace('\n', ' ')
    # get rid of spaces
    s = " ".join(s.split())
    list_sentence = s.split(' ')
    
    if left_window:
        if index > window:
            l = index - window
        else:
            l = 0

        return list_sentence[l:index]    
    else:
        mid_window = window // 2

        if index > mid_window:
            l = index - mid_window
        else:
            l = 0

        r = index + mid_window

        return list_sentence[l:r+1]
    
def unique(iter):
    "removes duplicates from iterable preserving order"
    result = list()
    seen = set()
    for x in iter:
        if x not in seen:
            seen.add(x)
            result.append(x)
    return result

def process_candidates(candidates, target):
    """ words to lower case, replace underscores, remove duplicated words, 
        filter out target word and stop words """
    filterwords = stopwords + [target]
    return unique(filter(lambda x : x not in filterwords, 
                  map(lambda s : s.lower().replace('_', ' '), candidates)))
    

# -- Build context -- #
context = defaultdict(lambda: list())

with open(os.path.join(DATA,'input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        key_index = int(fields[0])
        str_sentence = fields[1].strip()
        list_sentence = str_sentence.split()
        key = list_sentence[key_index]
        
        context[key] += process_candidates(unique(get_words(str_sentence, key_index, window=5)), key)

# for k, v in context.items():
#     print(k, v)

In [9]:
# -- Create model from the context instead of lexicon -- #

output_vector_filename = "glove.6B.100d.retrofit.similar10.context.{}"

generate_filename = DATA + '/' + output_vector_filename.format("magnitude")

# Use the Glove file to retrofit on
wv = Magnitude(DATA + "/glove.6B.100d.magnitude")
vocabQHat = set([k for k, v in wv])

isNumber = re.compile(r'\d+.*')

def norm_word(word):
    if isNumber.search(word.lower()):
        return '---num---'
    elif re.sub(r'\W+', '', word) == '':
        return '---punc---'
    else:
        return word.lower()

def copyPymagnitude(wv):
    return deepcopy({k: v for k, v in wv})

def get_substitutability(t, s):
    score = wv.similarity(t, s)
    return score

T = 10

qHat = copyPymagnitude(wv)
q = copyPymagnitude(wv)
loopOn = vocabQHat.intersection(set(context.keys()))

for t in range(T):
    for word in loopOn:
        wordNeighbours = set(context[word]).intersection(vocabQHat)
        numNeighbours = len(wordNeighbours)
        
        if numNeighbours == 0: 
            continue
            
        newVec = numNeighbours * qHat[word]
        
        for ppWord in wordNeighbours:
            newVec += q[ppWord]
            
        q[word] = newVec / (2*numNeighbours)

# -- Retrofitting based on cosine -- #
        
# for t in range(T):
#     for word in loopOn:
#         wordNeighbours = set(context[word]).intersection(vocabQHat)
#         cand_scores = []
#         for x in wordNeighbours:
#             cand_scores.append(get_substitutability(x, word))

#         sorted_candidates = sorted(zip(list(wordNeighbours), cand_scores), key=lambda x: x[1], reverse=True)
#         wordNeighbours = [sub for sub, score in sorted_candidates][:20]

#         numNeighbours = len(wordNeighbours)

#         if numNeighbours == 0:
#             continue

#         newVec = numNeighbours * qHat[word]

#         for ppWord in wordNeighbours:
#             newVec += q[ppWord]

#         q[word] = newVec / (2 * numNeighbours)

In [10]:
# generate text file
output_vector_text_file = DATA + "/" + output_vector_filename.format("txt")
with open(output_vector_text_file, "w") as f:
    for i, (k, v) in enumerate(q.items()):
        line = " ".join([k] + list(map(str, v)) + ["\n"])
        f.write(line)

print("Magnitude file will be saved at %sb" % generate_filename )        
        
# generate the magnitude file
subprocess.run(["python3", "-m", "pymagnitude.converter", "-i", output_vector_text_file, "-o", generate_filename])

Magnitude file will be saved at ../data/glove.6B.100d.retrofit.similar10.context.magnitudeb


CompletedProcess(args=['python3', '-m', 'pymagnitude.converter', '-i', '../data/glove.6B.100d.retrofit.similar10.context.txt', '-o', '../data/glove.6B.100d.retrofit.similar10.context.magnitude'], returncode=0)

#### Lexical substitution with context

In [11]:
class LexSub:

    def __init__(self, wvec_file, cvec_file=None, topn=10, n_candidates=15, score_type="Mult", window=5, left_window=True):
        self.wvecs = pymagnitude.Magnitude(wvec_file)
        self.cvecs = self.wvecs if not cvec_file else pymagnitude.Magnitude(cvec_file)
        self.n_candidates = n_candidates
        self.n_substitutes = topn
        self.score_type = score_type
        self.window = window
        self.left_window = left_window

    def substitutes(self, index, sentence):
        "Return ten guesses that are appropriate lexical substitutions for the word at sentence[index]."
        sentence_list = sentence.split(" ")
        word = sentence_list[index] 
        words_scores = self.wvecs.most_similar(word, topn=self.n_candidates)  # Get n possible replaceables
        result = [word for word, score in words_scores]
        result = process_candidates(result, word)
        return result
    
    def get_substitutability(self, t, s, C, score_type="Add"):
        if score_type == "Add":
            tscore = self.wvecs.similarity(s, t)
            cscores = [self.cvecs.similarity(s, c) for c in C]
            cscore = sum(cscores)
            return (tscore + cscore)/(len(C) + 1)
        elif score_type == "Mult":
            pcos = lambda a, b, vecs: (vecs.similarity(a, b) + 1) / 2
            
            pcos_s_t = pcos(s, t, self.wvecs)
            pcos_s_c_scores = [pcos(s, c, self.cvecs) for c in C]
            pcos_s_c = reduce(lambda x, y: x * y, pcos_s_c_scores, 1)
            
            result = (pcos_s_t * pcos_s_c)**(1./(len(C)+1))
            return result
            
    def lex_sub(self, index, sentence):
        """ Get appropriate substitution for a word given context words 
        
        word_POS = word with part of speech in form word.POS e.g. dog.n
        context_words = list of words in context 
        """
        # get list sentence. This will include punctuations and we cannot remove it because the index won't map
        list_ = sentence.split(" ")
        
        logger.debug("listSentence:%s", list_)
        
        w = list_[index]

        # generate candidate substitutions
        candidates = self.substitutes(index, sentence)
        
        logger.debug("word:%s, cand:%s", w, candidates)
        
        if sentence is None:
            return candidates[:self.n_substitutes]
        else:
            # This returns a list of words from the sentence on a N sized window
            context_words = get_words(sentence, index=index, window=self.window, left_window=self.left_window)
            
            logger.debug("word:%s, unFiltered->%s", w, context_words)
            
            # Make sure that these context words are present in our retroffitted vector
            # They are not stop words
            # And they are not equal to the target word
            context_words = list(filter(lambda c : c in self.wvecs 
                                              and c not in stopwords and c != w, 
                                              context_words))
            if not context_words:
                logger.debug("sent:%s, context->%s", sentence, context_words)
                
            logger.debug("word:%s, context->%s", w, context_words)
            
            cand_scores = [self.get_substitutability(w, s, context_words, score_type=self.score_type) for s in candidates]
            assert(len(cand_scores) == len(candidates))            
            sorted_candidates = sorted(zip(candidates, cand_scores), key = lambda x : x[1], reverse=True )
            logger.debug("word:%s, sortedCand:%s", w, sorted_candidates[:self.n_substitutes])
            return [sub for sub, score in sorted_candidates][:self.n_substitutes]

# Run program
logger.setLevel(logging.INFO)
lexsub = LexSub(
    os.path.join(DATA,'glove.6B.100d.retrofit.similar10.magnitude'),
    os.path.join(DATA,'glove.6B.100d.retrofit.similar10.context.magnitude'),
    n_candidates=12,
    score_type="Add",
    window=5,
    left_window=False
)
output = []
keys = []

# Run on the retrofit file
with open(os.path.join(DATA,'input','dev.txt')) as f:
    for line in f:
        fields = line.strip().split('\t')
        index = int(fields[0].strip())
        list_sentence = fields[1].strip().split(" ")
        str_sentence = fields[1].strip()
        
        keys.append(list_sentence[index])
        
        output.append(" ".join(lexsub.lex_sub(int(fields[0]), str_sentence)))
        
for i, syn in enumerate(output[:20]):
    print("target:%s" % keys[i], "substitutes:%s" % output[i])

# Check score
with open(os.path.join(DATA,'reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("\nScore={:.2f}".format(100*precision(ref_data, output)))

target:side substitutes:bottom corner back place left edge front line right way
target:side substitutes:way part right place back line front edge bottom left
target:side substitutes:way back part right place line left front edge bottom
target:side substitutes:front bottom back edge line corner left place right way
target:side substitutes:way place right part back front left line corner edge
target:side substitutes:front bottom back edge line corner left place right way
target:side substitutes:place way right part back line bottom front left corner
target:side substitutes:front bottom back edge line corner left place right way
target:side substitutes:way right line front part back place edge bottom left
target:side substitutes:part line place corner edge way back front left right
target:told substitutes:reporters asked interview afp quoted insisted spokesman press statement spoke
target:tell substitutes:let say see think know answer ask hear remember guess
target:tell substitutes:know a

#### Hence we take the initial retrofitted model as our final model with a score of 43.57